In [22]:
import torch
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import os
import numpy as np
import json
import csv
from PIL import Image
from tqdm import tqdm
import string
import time

In [3]:
# th architecture to use
arch = 'resnet50'
# resnet50
# resnet18
# alexnet

# load the pre-trained weights
model_file = '%s_places365.pth.tar' % arch
if not os.access(model_file, os.W_OK):
    weight_url = 'http://places2.csail.mit.edu/models_places365/' + model_file
    os.system('wget ' + weight_url)

model = models.__dict__[arch](num_classes=365)
checkpoint = torch.load(model_file, map_location=lambda storage, loc: storage)
state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
model.load_state_dict(state_dict)
model.eval()

# load the image transformer
centre_crop = trn.Compose([
        trn.Resize((256,256)),
        trn.CenterCrop(224),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


# load the class label
file_name = 'categories_places365.txt'
if not os.access(file_name, os.W_OK):
    synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
    os.system('wget ' + synset_url)
classes = list()
with open(file_name) as class_file:
    for line in class_file:
        classes.append(line.strip().split(' ')[0][3:])
classes = tuple(classes)

# load the test image
img_name = '12.jpg'
if not os.access(img_name, os.W_OK):
    img_url = 'http://places.csail.mit.edu/demo/' + img_name
    os.system('wget ' + img_url)

img = Image.open(img_name)
input_img = V(centre_crop(img).unsqueeze(0))

In [4]:
# forward pass
logit = model.forward(input_img)
h_x = F.softmax(logit, 1).data.squeeze()
probs, idx = h_x.sort(0, True)

print('{} prediction on {}'.format(arch,img_name))
# output the prediction
for i in range(0, 5):
    print('{:.3f} -> {}'.format(probs[i], classes[idx[i]]))

resnet50 prediction on 12.jpg
0.685 -> patio
0.240 -> restaurant_patio
0.019 -> beer_garden
0.010 -> courtyard
0.010 -> porch


# Evaluate Pre-trained Models

In [207]:
# evaluation
backbone_network = 'alexnet'
val_path = '../../places365_data/val_large/'
val_data = os.listdir(val_path)
val_data.sort()
# model = VGG16_Places365(weights='places')

# def save_prob_idx (prob,idx):
#     sample = {}
#     for i in range(int(prob.size()[0])):
#         sample[int(idx[i])]= float(prob[i])
#     return sample

def validation(arch):
    arch = 'alexnet'
# resnet50, resnet18, alexnet
    model_file = '%s_places365.pth.tar' % arch
    print(model_file)
    
    if not os.access(model_file, os.W_OK):
        weight_url = 'http://places2.csail.mit.edu/models_places365/' + model_file
        os.system('wget ' + weight_url)

    model = models.__dict__[arch](num_classes=365)
    model.cuda()
    checkpoint = torch.load(model_file, map_location=lambda storage, loc: storage)
    state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
    model.load_state_dict(state_dict)
    model.eval()
    
    centre_crop = trn.Compose([
            trn.Resize((256,256)),
            trn.CenterCrop(224),
            trn.ToTensor(),
            trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    # load categories
    file_name = 'categories_places365.txt'
    if not os.access(file_name, os.W_OK):
        synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
        os.system('wget ' + synset_url)
    classes = list()
    with open(file_name) as class_file:
        for line in class_file:
            classes.append(line.strip().split(' ')[0][3:])
    classes = tuple(classes)

    # start to val
    val_res = []
    count = 0
    for data in tqdm(val_data):
        image = Image.open(os.path.join(val_path,data)).convert('RGB') 
        input_img = V(centre_crop(image).unsqueeze(0)).cuda()
        # forward pass
        logit = model.forward(input_img)
#         print(logit)
        h_x = F.softmax(logit, 1).data.squeeze()
#         print(h_x)
#         probs, idx = h_x.sort(0, True)
        name = data[10:-4]
#         dt_y = save_prob_idx(probs,idx)
        dt_y = h_x.tolist()
        dt_y.insert(0,name)
        val_res.append(dt_y)

    return val_res



val = validation(backbone_network)

print('total {} samples'.format(len(val)))

# Write into a csv file

with open('val_results_{}.csv'.format(backbone_network),'w') as f:
    writer = csv.writer(f)
    for row in val:
        writer.writerow(row)

# Read the preditctions
with open('val_results_{}.csv'.format(backbone_network),"r") as csvfile:
    reader = csv.reader(csvfile)
    dts = []
    for line in reader:
        dts.append(line)
print('total {} samples'.format(len(dts)))

alexnet_places365.pth.tar


  7%|▋         | 2683/36500 [00:34<07:08, 78.85it/s]


KeyboardInterrupt: 

# Real Validation

In [137]:
# load the class label
file_name = 'categories_places365.txt'
if not os.access(file_name, os.W_OK):
    synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
    os.system('wget ' + synset_url)
classes = list()
with open(file_name) as class_file:
    for line in class_file:
        classes.append(line.strip().split(' ')[0][3:])
classes = tuple(classes)

# load ground truth
with open ('../../places365_data/annotations/places365_val.txt','r') as f:
    gts = f.readlines()
gts_x = [i.split()[0][10:-4] for i in gts]
gts_y = [i.split()[1] for i in gts]
gts = {}
for i in range(len(gts_x)):
    gts[gts_x[i]] = int(gts_y[i])
gt = [int(a) for a in list(gts.values())]


def top1_prediction (dt):
#     print(len(dt))
    new_dt = [float(i) for i in dt]
    m = max(new_dt)
#     print(m)
    top1 = [i for i, j in enumerate(new_dt) if j == m]
    return top1

def top1_accuracy(dts,gts):
    total = 0
    accurate = 0
    for key in dts.keys():
        if dts[key] == gts[key]:
            accurate +=1
        total += 1
    return accurate/total


In [148]:
backbone_network = 'resnet50'
with open('val_results_{}.csv'.format(backbone_network),"r") as csvfile:
    reader = csv.reader(csvfile)
    dts = []
    for line in reader:
        dts.append(line)
# print('total {} samples'.format(len(dts)))

dts_top1 = {}
for dt in dts:
    dts_top1[dt[0]] = top1_prediction(dt[1:])[0]

In [149]:

    

top1 = top1_accuracy(dts_top1,gts)
print('Top-1 accuracy of {} is {}'.format(backbone_network,float(top1)))


# Top-1 accuracy of resnet50 is 0.5508767123287671
# Top-1 accuracy of alexnet is 0.4766849315068493
# Top-1 accuracy of resnet18 is 0.5385753424657534

Top-1 accuracy of resnet50 is 0.5508767123287671


# Validate at higher levels

In [194]:
# level 1

import csv
level1 = {}
with open('../places365_level1.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        level1[int(row[0])]=[int(row[1]),int(row[2]),int(row[3])]    
        
temp = []
for label in level1.keys():
    if level1[label] not in temp:
        temp.append(level1[label])
lv1_classes = {}
for i in range(len(temp)):
    lv1_classes[i] = temp[i]
print(lv1_classes)

lv1 = {}
for key in level1.keys():
    lv1[key] = temp.index(level1[key])
# lv1 mapping original class_id to level1_id  

def convert_dts2level1 (dt):
#     print(len(dt))
    dt_new={}
    for key in lv1_classes.keys():
        dt_new[key] = 0
    for i in range(len(dt)):
        dt_new[lv1[i]] = float(dt[i]) + float(dt_new[lv1[i]])
    flag = 0
    temp = 0 
    for key in dt_new.keys():
        if dt_new[key] > temp:
            flag = key
            temp  = dt_new[key]
        
    return flag

{0: [0, 0, 1], 1: [1, 0, 0], 2: [0, 1, 1], 3: [0, 1, 0], 4: [1, 0, 1]}


In [195]:
# mapping gts to level1


# load the class label
file_name = 'categories_places365.txt'
if not os.access(file_name, os.W_OK):
    synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
    os.system('wget ' + synset_url)
classes = list()
with open(file_name) as class_file:
    for line in class_file:
        classes.append(line.strip().split(' ')[0][3:])
classes = tuple(classes)

# load ground truth
with open ('../../places365_data/annotations/places365_val.txt','r') as f:
    gts = f.readlines()
gts_x = [i.split()[0][10:-4] for i in gts]
gts_y = [i.split()[1] for i in gts]
gts = {}
for i in range(len(gts_x)):
    gts[gts_x[i]] = int(gts_y[i])

gts_lv1 = {}
for key in gts.keys():
    value = gts[key]
    gts_lv1[key] = lv1[value]



In [200]:
# mapping dts to level1:


backbone_network = 'alexnet'
with open('val_results_{}.csv'.format(backbone_network),"r") as csvfile:
    reader = csv.reader(csvfile)
    dts = []
    for line in reader:
        dts.append(line)

dts_lv1 = {}
for dt in dts:
    dts_lv1[dt[0]] = convert_dts2level1(dt[1:])
    
    
top1 = top1_accuracy(dts_lv1,gts_lv1)
print('Top-1 accuracy of {} is {}'.format(backbone_network,float(top1)))

# Top-1 accuracy of resnet50 is 0.8876986301369864
# Top-1 accuracy of resnet18 is 0.884027397260274
# Top-1 accuracy of alexnet is 0.8622739726027397


Top-1 accuracy of alexnet is 0.8622739726027397


In [ ]:
# level 2:

In [202]:
# level 1

import csv
level2 = {}
with open('../places365_level2.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        level2[int(row[0])]=[row[a] for a in range(1,len(row))]
        
temp = []
for label in level2.keys():
    if level2[label] not in temp:
        temp.append(level2[label])
lv2_classes = {}
for i in range(len(temp)):
    lv2_classes[i] = temp[i]
print(lv2_classes)

lv2 = {}
for key in level2.keys():
    lv2[key] = temp.index(level2[key])
# lv1 mapping original class_id to level1_id  

def convert_dts2level2 (dt):
#     print(len(dt))
    dt_new={}
    for key in lv2_classes.keys():
        dt_new[key] = 0
    for i in range(len(dt)):
        dt_new[lv2[i]] = float(dt[i]) + float(dt_new[lv2[i]])
    flag = 0
    temp = 0 
    for key in dt_new.keys():
        if dt_new[key] > temp:
            flag = key
            temp  = dt_new[key]
        
    return flag

{0: ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0'], 1: ['0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], 2: ['0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], 3: ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1'], 4: ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0'], 5: ['0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], 6: ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0'], 7: ['0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], 8: ['0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0'], 9: ['0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], 10: ['1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], 11: ['0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0'

In [203]:
# mapping gts to level2


# load the class label
file_name = 'categories_places365.txt'
if not os.access(file_name, os.W_OK):
    synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
    os.system('wget ' + synset_url)
classes = list()
with open(file_name) as class_file:
    for line in class_file:
        classes.append(line.strip().split(' ')[0][3:])
classes = tuple(classes)

# load ground truth
with open ('../../places365_data/annotations/places365_val.txt','r') as f:
    gts = f.readlines()
gts_x = [i.split()[0][10:-4] for i in gts]
gts_y = [i.split()[1] for i in gts]
gts = {}
for i in range(len(gts_x)):
    gts[gts_x[i]] = int(gts_y[i])

gts_lv2 = {}
for key in gts.keys():
    value = gts[key]
    gts_lv2[key] = lv2[value]


In [208]:
# mapping dts to level1:


backbone_network = 'resnet18'
with open('val_results_{}.csv'.format(backbone_network),"r") as csvfile:
    reader = csv.reader(csvfile)
    dts = []
    for line in reader:
        dts.append(line)

dts_lv2 = {}
for dt in dts:
    dts_lv2[dt[0]] = convert_dts2level2(dt[1:])
    
    
top1 = top1_accuracy(dts_lv2,gts_lv2)
print('Top-1 accuracy of {} is {}'.format(backbone_network,float(top1)))


# Top-1 accuracy of resnet50 is 0.7248767123287672
# Top-1 accuracy of resnet18 is 0.7152876712328767
# Top-1 accuracy of alexnet is 0.6586301369863014

Top-1 accuracy of resnet18 is 0.7152876712328767
